In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
# Function to scrape Amazon
def scrape_amazon(query, max_results=5):
    headers = {"User-Agent": "Mozilla/5.0"}
    url = f"https://www.amazon.in/s?k={query.replace(' ', '+')}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    products = []
    results = soup.find_all("div", {"data-component-type": "s-search-result"})

    for item in results[:max_results]:
        name = item.h2.text.strip() if item.h2 else "N/A"
        price = item.find("span", "a-price-whole")
        price = price.text.strip() if price else "N/A"
        rating = item.find("span", "a-icon-alt")
        rating = rating.text.strip() if rating else "N/A"

        products.append(["Amazon", name, price, rating])
    return products

In [31]:
# Function to scrape Flipkart
def scrape_flipkart(query, max_results=5):
    headers = {"User-Agent": "Mozilla/5.0"}
    url = f"https://www.flipkart.com/search?q={query.replace(' ', '+')}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    products = []
    results = soup.find_all("div", {"class": "_1AtVbE"})

    for item in results:
        name = item.find("div", {"class": "_4rR01T"}) or item.find("a", {"class": "IRpwTa"})
        if not name:
            continue

        price = item.find("div", {"class": "_30jeq3"})
        rating = item.find("div", {"class": "_3LWZlK"})

        products.append(["Flipkart",
                         name.text.strip() if name else "N/A",
                         price.text.strip() if price else "N/A",
                         rating.text.strip() if rating else "N/A"])

        if len(products) >= max_results:
            break
    return products

In [25]:
# Store in file
if __name__ == "__main__":
    query = input("Enter the product to search: ")

    amazon_data = scrape_amazon(query)
    flipkart_data = scrape_flipkart(query)

    # Combine both data
    all_data = amazon_data + flipkart_data

    # Saving
    df = pd.DataFrame(all_data, columns=["Website", "Product Name", "Price", "Rating"])
    df.to_excel("product_details.xlsx", index=False)

    print("Data saved to product_details.xlsx")

Enter the product to search:  phone


Data saved to product_details.xlsx
